In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U langchain
!pip install -q -U ctransformers[cuda]
!pip install chromadb
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [3]:
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
llm = HuggingFacePipeline(pipeline=pipeline)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
model_id = "mistralai/Mistral-7B-Instruct-v0.1"


model_4bit = AutoModelForCausalLM.from_pretrained( model_id, device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=1000,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [10]:
#### Prompt
template = """[INST] You are a helpful, respectful and honest assistant.
Answer the question below:
{question} [/INST]
"""

question_p = """What is the use of hiletgo gy-521 module in an IoT kit?"""
# context_p = """ On August 10 said that its arm JSW Neo Energy has agreed to buy a portfolio of 1753 mega watt renewable energy generation capacity from Mytrah Energy India Pvt Ltd for Rs 10,530 crore."""
prompt = PromptTemplate(template=template, input_variables={"question"})
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question":question_p})
response

TypeError: unhashable type: 'list'

In [13]:
text = "What is the use of hiletgo gy-521 module in an IoT kit?"

device = 'cuda'
model_inputs = tokenizer(text, return_tensors="pt").to(model_4bit.device)

generated_ids = model_4bit.generate(**model_inputs, temperature=0.1, top_k=1, top_p=1.0, repetition_penalty=1.4, min_new_tokens=16, max_new_tokens=500, do_sample=True)
decoded = tokenizer.decode(generated_ids[0])
print(decoded)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> What is the use of hiletgo gy-521 module in an IoT kit?
HILETGO Gy-521 Module: The HIL (Hardware In Loop) Testing System for Automotive and Aerospace Applications. It provides a flexible, cost-effective solution to test automotive control systems under real conditions without risking human life or damaging expensive equipment. With its advanced features such as high accuracy, low latency, and support for multiple protocols, it enables engineers to validate complex system behavior quickly and efficiently.</s>


In [15]:
text = "What is the use of RC522 RFID Module in an IoT kit? Write the description, applications and technical specifications"

device = 'cuda'
model_inputs = tokenizer(text, return_tensors="pt").to(model_4bit.device)

generated_ids = model_4bit.generate(**model_inputs, temperature=0.1, top_k=1, top_p=1.0, repetition_penalty=1.4, min_new_tokens=16, max_new_tokens=2000, do_sample=True)
decoded = tokenizer.decode(generated_ids[0])
print(decoded)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> What is the use of RC522 RFID Module in an IoT kit? Write the description, applications and technical specifications.

Description: The RS-483 module (also known as Modbus) allows for communication between devices using a serial protocol over twisted pair cables or optical fibers. It uses differential signaling to transmit data at high speeds with low noise levels. This makes it ideal for industrial automation systems where reliability and accuracy are critical factors. Some common applications include process control, building management systems, HVAC controls, and machine monitoring. Technical Specifications: - Baud rate range from 19600 bps up to 115200 bps - Data frame length ranging from 7 bytes to 1023 bytes - Parity options available including none, odd parity, even parity, and checksums - Differential encoding used for signal transmission - Compatible with most microcontrollers and computers</s>


In [16]:
text = "What is the use of Servo Motor SG90 in an IoT kit? Write the description, applications and technical specifications"

device = 'cuda'
model_inputs = tokenizer(text, return_tensors="pt").to(model_4bit.device)

generated_ids = model_4bit.generate(**model_inputs, temperature=0.1, top_k=1, top_p=1.0, repetition_penalty=1.4, min_new_tokens=16, max_new_tokens=2000, do_sample=True)
decoded = tokenizer.decode(generated_ids[0])
print(decoded)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> What is the use of Servo Motor SG90 in an IoT kit? Write the description, applications and technical specifications.

SG-91 servos are a popular choice for hobbyists due to their low cost and ease of control with Arduino or other microcontrollers. They have 6V DC motors that can be controlled by sending pulses from your controller's digital pins. The range of motion on these servos is typically around 270 degrees (from fully extended to almost touching), but they can also rotate all the way back up to about 35 degrees past center position if you need them to do so. This makes it easy to create robotic arms or moving platforms using just one motor.</s>


In [17]:
text = "What is the use of Stepper Motor in an IoT kit? Write the description, applications and technical specifications"

device = 'cuda'
model_inputs = tokenizer(text, return_tensors="pt").to(model_4bit.device)

generated_ids = model_4bit.generate(**model_inputs, temperature=0.1, top_k=1, top_p=1.0, repetition_penalty=1.4, min_new_tokens=16, max_new_tokens=2000, do_sample=True)
decoded = tokenizer.decode(generated_ids[0])
print(decoded)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> What is the use of Stepper Motor in an IoT kit? Write the description, applications and technical specifications.
Stepper Motors are widely used for various purposes such as 3D printing, CNC machines, robotics etc., In this article we will discuss about its usage in Internet Of Things (IoT) kits. A stepper motor converts electrical pulses into mechanical motion by moving a series of electromagnets along fixed tracks or coils to create precise movements with high torque-to-weight ratio. It has become increasingly popular due to its ability to provide accurate positioning control over small distances while consuming minimal power compared to other motors like DC motors. Here's how it works: The motor consists of multiple poles arranged around a central shaft that rotates on bearings at constant speed. Each pole contains one or more permanent magnets which interact with corresponding electromagnetic fields generated by current flowing through wires wrapped around each pole. This inter

In [18]:

text = "What is the use of Water Level Detection Sensor Module in an IoT kit? Write the description, applications and technical specifications"

device = 'cuda'
model_inputs = tokenizer(text, return_tensors="pt").to(model_4bit.device)

generated_ids = model_4bit.generate(**model_inputs, temperature=0.1, top_k=1, top_p=1.0, repetition_penalty=1.4, min_new_tokens=16, max_new_tokens=2000, do_sample=True)
decoded = tokenizer.decode(generated_ids[0])
print(decoded)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> What is the use of Water Level Detection Sensor Module in an IoT kit? Write the description, applications and technical specifications.
Water level detection sensor module helps to measure water levels accurately by using ultrasonic technology or capacitive sensing method. This type of sensors are widely used for various purposes such as monitoring liquid levels in tanks, reservoirs, wells etc. The main advantage of this sensor over other traditional methods like dipsticks is that it provides a non-invasive way to check the fluid levels without physically touching them which can cause contamination. 
Applications:
1. Agriculture - Farmers can monitor soil moisture levels with these sensors to optimize irrigation systems and reduce wastage.
2. Industrial Processes - These sensors help industries to maintain optimal operating conditions in their processes such as chemical reactions, cooling towers, boilers etc.
3. Home Automation - Smart homes can be equipped with these sensors to co